In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
import hvplot.pandas
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
tide_file = '/home/jovyan/data/tide/pred_F.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
df_tides= df_tides.sort_index()
#df_tides_E['height'] = df_tides_E['height'].to_pandas().resample('T').mean()
df_tides.head

In [ ]:
df_tides.describe()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]

In [ ]:
df_tides['epoch'] = epoch

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as F:
    botpt_data = pk.load(F)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['Depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()
df_botpt.index.name= 'Date'
df_botpt.tail()

In [ ]:
df_botpt.hvplot.scatter(x='Date', y='Depth', height=200, datashade=True,
                       flip_yaxis=True)

In [ ]:
df_subsetTides = df_tides.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']
df_subsetBOTPT = df_botpt.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']

In [ ]:
type(df_botpt)

In [ ]:
df_subsetBOTPT['tides'] = df_subsetTides.height
df_subsetBOTPT.tail()

In [ ]:
df_subsetBOTPT['dtide']=  df_subsetBOTPT['Depth'] - df_subsetBOTPT['tides']
df_subsetBOTPT.tail()

In [ ]:
df_subsetBOTPT.hvplot(x='Date', y='dtide', datashade=True, height=200,
                       flip_yaxis=True)

### Import Seismic Data

In [ ]:
seismic_file = '/home/jovyan/data/hypo71.dat.txt'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2019-06-17 00:00:00']
df_seismic_data.head()
# del df_seismic_data['yyyymmdd']
# del df_seismic_data['HHMM']
# del df_seismic_data['SSS.SS']
# del df_seismic_data['Depth']
# del df_seismic_data['MW']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']
del df_seismic_data['Lat(D']
del df_seismic_data['Lon(D']
del df_seismic_data['M)']
del df_seismic_data['M).1']
del df_seismic_data['PMom']
del df_seismic_data['SMom']
# del df_seismic_data['RMS']
df_seismic_data.tail()

1. yyyymmdd - year, month, day
2. HHMM - hour, minute
3. Lat(D - latitude degree 
4. M) - latitude minute 
5. Lon(D - longitude degreee
6. M).1 - longitude minute
7. Depth - depth in km
8. MW - moment magnitude (based on the concept of seismic moment) - rupture area, slip 
9. NWR - NOAA Weather Radio???
10. GAP - seismic gap (section of a fault that has produced earthquakes in the past but is now quiet) - azimuthal 
11. DMIN - Horizontal distance from the epicenter to the nearest station (in km). In general, the smaller this number, the more reliable is the calculated depth of the earthquake
12. RMS - root mean square 
13. ERH - error estimates in horizontal direction 
14. ERZ - error estimates in vertical direction
15. ID - measurement ID???
16. PMom - body wave magnitude?
17. SMom - surface wave magnitude?

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
df_seismic_data.tail()

In [ ]:
df_seismic_data.hvplot(x='datetime', y='MW', kind='scatter')

#### Richter magnitude (Local Magnitude ML)
This is still used for measuring the magnitude of shallow events at distances less than 600 km 
<br/>ML = logA + 2.56logD - 1.67
<br/>A: measured ground motion (in micrometres)
<br/>D: distance from the event (in km)

### What is A??

#### Body wave magnitude Mb
For earthquakes measured at distances greater than 600 km magnitude can be estimated from the formula.
<br/> Mb = log(A/T) + σ(D,h)

A: maximum amplitude (in micrometres)of the P-waves measured at period T (generally about one second) 
<br/> σ: calibration term (in the range 6–8) that depends on distance from the event D and depth of the event h (tables of σ are used).

#### Surface wave magnitude Ms
For shallow earthquakes (i.e. ones that generate surface waves) magnitudes can be estimated using the formula.

MS = log(A/T) + 1.66logΔ + 3.3

A: maximum amplitude (in micrometres) of the Rayleigh waves
<br/>T: period (usually about 20 seconds) 
<br/>Δ: distance (in degrees).

#### Moment magnitude
Earthquakes occur when the ground ruptures. Stresses build up over time (usually caused by the slow movements of tectonic plates) and eventually a piece of the Earth's brittle crust deep under ground breaks (the technical term is ruptures). This rupture then grows until eventually a large area has shifted (the rupture propagates at a velocity of 2–3km/sec). The magnitude of the earthquake is related to the size of the rupture.

Seismic moment (Mo) = μ* rupture area * slip length

μ: shear modulus of the crust (approx 3x1010 N/m)

Moment magnitude (Mw) = 2/3log(Mo) - 6.06



#### An earthquake can have a negative magnitude
Very small events (e.g. If 2/3log(Mo)<6.0 or if logA+2.56logD<1.67 ) will have a magnitude less than zero. In practice, earthquakes this small, although quite numerous, are usually only recorded and located in very small scale studies (e.g. studying rockbursts in underground mines).